In [3]:
from flask import Flask, request, jsonify
from transformers import BartForConditionalGeneration, BartTokenizer
from flask_cors import CORS

In [4]:

app = Flask(__name__)
CORS(app)

In [5]:

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [6]:
def summarize_text(text, max_length=150):
    inputs = tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, max_length=max_length, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return summary

In [7]:
@app.route("/api/answer", methods=["POST"])
def get_answer():
    data = request.get_json()
    text = data.get("text", "")
    question = data.get("question", "")

    if question:
        inputs = tokenizer(question, text, return_tensors="pt", max_length=512, truncation=True, padding=True)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, num_beams=4, max_length=128, early_stopping=True)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        return jsonify({"question": question, "answer": answer})
    else:
        summary = summarize_text(text)
        return jsonify({"summary": summary})

In [8]:

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
Press CTRL+C to quit
